# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 13 2022 9:17AM,246227,15,30010579,"Alliance Pharma, Inc.",Released
1,Sep 13 2022 9:17AM,246227,15,30010580,"Alliance Pharma, Inc.",Released
2,Sep 13 2022 9:17AM,246227,15,30010581,"Alliance Pharma, Inc.",Released
3,Sep 13 2022 9:17AM,246227,15,30010582,"Alliance Pharma, Inc.",Released
4,Sep 13 2022 9:17AM,246227,15,30010584,"Alliance Pharma, Inc.",Released
5,Sep 13 2022 9:17AM,246227,15,30010583,"Alliance Pharma, Inc.",Released
6,Sep 13 2022 9:17AM,246227,15,30010586,"Alliance Pharma, Inc.",Released
7,Sep 13 2022 9:17AM,246227,15,30010585,"Alliance Pharma, Inc.",Released
8,Sep 13 2022 9:17AM,246227,15,30010587,"Alliance Pharma, Inc.",Released
9,Sep 13 2022 9:17AM,246227,15,30010588,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,246248,Executing,3
30,246248,Released,2
31,246249,Released,1
32,246251,Released,1
33,246252,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246245,NaN,NaN,1.0
246248,NaN,3.0,2.0
246249,NaN,NaN,1.0
246251,NaN,NaN,1.0
246252,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246062,0.0,0.0,1.0
246063,0.0,0.0,1.0
246064,0.0,0.0,1.0
246156,0.0,1.0,4.0
246165,0.0,0.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246062,0,0,1
246063,0,0,1
246064,0,0,1
246156,0,1,4
246165,0,0,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246062,0,0,1
1,246063,0,0,1
2,246064,0,0,1
3,246156,0,1,4
4,246165,0,0,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246062,,,1
1,246063,,,1
2,246064,,,1
3,246156,,1,4
4,246165,,,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 13 2022 9:17AM,246227,15,"Alliance Pharma, Inc."
119,Sep 13 2022 8:53AM,246252,19,VITABIOTICS USA INC.
120,Sep 13 2022 8:50AM,246251,19,VITABIOTICS USA INC.
121,Sep 13 2022 8:36AM,246248,20,Alumier Labs Inc.
126,Sep 13 2022 8:36AM,246249,10,Beach Patient Assistance
127,Sep 13 2022 8:35AM,246244,10,ISDIN Corporation
128,Sep 13 2022 8:35AM,246239,10,ISDIN Corporation
196,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc."
201,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation
210,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 13 2022 9:17AM,246227,15,"Alliance Pharma, Inc.",,,119
1,Sep 13 2022 8:53AM,246252,19,VITABIOTICS USA INC.,,,1
2,Sep 13 2022 8:50AM,246251,19,VITABIOTICS USA INC.,,,1
3,Sep 13 2022 8:36AM,246248,20,Alumier Labs Inc.,,3,2
4,Sep 13 2022 8:36AM,246249,10,Beach Patient Assistance,,,1
5,Sep 13 2022 8:35AM,246244,10,ISDIN Corporation,,,1
6,Sep 13 2022 8:35AM,246239,10,ISDIN Corporation,,,68
7,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",,,5
8,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,,3,6
9,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 9:17AM,246227,15,"Alliance Pharma, Inc.",119,,
1,Sep 13 2022 8:53AM,246252,19,VITABIOTICS USA INC.,1,,
2,Sep 13 2022 8:50AM,246251,19,VITABIOTICS USA INC.,1,,
3,Sep 13 2022 8:36AM,246248,20,Alumier Labs Inc.,2,3,
4,Sep 13 2022 8:36AM,246249,10,Beach Patient Assistance,1,,
5,Sep 13 2022 8:35AM,246244,10,ISDIN Corporation,1,,
6,Sep 13 2022 8:35AM,246239,10,ISDIN Corporation,68,,
7,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",5,,
8,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,6,3,
9,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 9:17AM,246227,15,"Alliance Pharma, Inc.",119,,
1,Sep 13 2022 8:53AM,246252,19,VITABIOTICS USA INC.,1,,
2,Sep 13 2022 8:50AM,246251,19,VITABIOTICS USA INC.,1,,
3,Sep 13 2022 8:36AM,246248,20,Alumier Labs Inc.,2,3,
4,Sep 13 2022 8:36AM,246249,10,Beach Patient Assistance,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 9:17AM,246227,15,"Alliance Pharma, Inc.",119.0,NaN,NaN
1,Sep 13 2022 8:53AM,246252,19,VITABIOTICS USA INC.,1.0,NaN,NaN
2,Sep 13 2022 8:50AM,246251,19,VITABIOTICS USA INC.,1.0,NaN,NaN
3,Sep 13 2022 8:36AM,246248,20,Alumier Labs Inc.,2.0,3.0,NaN
4,Sep 13 2022 8:36AM,246249,10,Beach Patient Assistance,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 9:17AM,246227,15,"Alliance Pharma, Inc.",119.0,0.0,0.0
1,Sep 13 2022 8:53AM,246252,19,VITABIOTICS USA INC.,1.0,0.0,0.0
2,Sep 13 2022 8:50AM,246251,19,VITABIOTICS USA INC.,1.0,0.0,0.0
3,Sep 13 2022 8:36AM,246248,20,Alumier Labs Inc.,2.0,3.0,0.0
4,Sep 13 2022 8:36AM,246249,10,Beach Patient Assistance,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3447122,124.0,4.0,10.0
102,246194,1.0,0.0,0.0
15,246227,119.0,0.0,0.0
16,492370,4.0,2.0,0.0
18,246193,0.0,1.0,0.0
19,984925,5.0,0.0,0.0
20,492436,6.0,3.0,0.0
21,984357,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3447122,124.0,4.0,10.0
1,102,246194,1.0,0.0,0.0
2,15,246227,119.0,0.0,0.0
3,16,492370,4.0,2.0,0.0
4,18,246193,0.0,1.0,0.0
5,19,984925,5.0,0.0,0.0
6,20,492436,6.0,3.0,0.0
7,21,984357,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,124.0,4.0,10.0
1,102,1.0,0.0,0.0
2,15,119.0,0.0,0.0
3,16,4.0,2.0,0.0
4,18,0.0,1.0,0.0
5,19,5.0,0.0,0.0
6,20,6.0,3.0,0.0
7,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,124.0
1,102,Released,1.0
2,15,Released,119.0
3,16,Released,4.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,21
Status,,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,0.0,0.0,2.0,1.0,0.0,3.0,1.0
Released,124.0,1.0,119.0,4.0,0.0,5.0,6.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,0.0,2.0,1.0,0.0,3.0,1.0
2,Released,124.0,1.0,119.0,4.0,0.0,5.0,6.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,0.0,2.0,1.0,0.0,3.0,1.0
2,Released,124.0,1.0,119.0,4.0,0.0,5.0,6.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()